In [23]:
from mocpy import MOC
from PIL import Image
import numpy as np 
from ipyaladin import Aladin
import cdshealpix
from astropy import wcs
from astropy.coordinates import Longitude, Latitude
from astropy.coordinates import SkyCoord
import astropy.units as u
import cdshealpix
import requests
from io import BytesIO

depth = 10

In [40]:
im_url = 'https://www.acrola.fr/wp-content/uploads/2021/06/cigogne-atterissage-bis-1536x993.jpg'

response = requests.get(im_url)
im = Image.open(BytesIO(response.content))
im = np.fliplr(np.swapaxes(im,0,1))

In [41]:

w = wcs.WCS(naxis=2)

w.wcs.crpix = [im.shape[1], 0]
w.wcs.cdelt = np.array([-40/np.max(im.shape), 40/np.max(im.shape)])
w.wcs.crval = [0, 0]
w.wcs.ctype = ["GLON-SIN", "GLAT-SIN"]

In [42]:
white = np.where(im < 64)
gray_light = np.where((im>=64) & (im<=127))
gray_dark = np.where((im>=127) & (im<=192))
black = np.where(im>192)

In [43]:
def create_moc(world_coo, output):
    global depth
    
    sc = SkyCoord(world_coo[0], world_coo[1], unit='deg', frame='galactic')
    
    lon = Longitude(sc.icrs.ra.degree, u.deg)
    lat = Latitude(sc.icrs.dec.degree, u.deg)
    
    ipix = cdshealpix.lonlat_to_healpix(lon, lat, depth)
    
    
    moc = MOC.from_healpix_cells(ipix, np.repeat(depth, ipix.shape[0]), depth)
    
    return moc


In [44]:

world_white = w.wcs_pix2world(white[0], white[1], 1)
world_gray_light = w.wcs_pix2world(gray_light[0], gray_light[1], 1)
world_gray_dark = w.wcs_pix2world(gray_dark[0], gray_dark[1], 1)
world_black = w.wcs_pix2world(black[0], black[1], 1)

In [45]:
white = create_moc(world_white, 'white.fits')
gray_light = create_moc(world_gray_light, 'gray_light.fits')
gray_dark = create_moc(world_gray_dark, 'gray_dark.fits')
black = create_moc(world_black, 'black.fits')


In [56]:
a = Aladin(survey='Mellinger', coo_frame='galactic', fov= 50, target= "6 +12")
a

Aladin(coo_frame='galactic', fov=50.0, options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log'…

In [57]:
a.add_moc_from_dict(white.serialize("json"), {"color": "white", "opacity": 0.5})
a.add_moc_from_dict(black.serialize("json"), {"color": "black", "opacity": 0.5})
a.add_moc_from_dict(gray_light.serialize("json"), {"color": "lightGray", "opacity": 0.5})
a.add_moc_from_dict(gray_dark.serialize("json"), {"color": "darkGray", "opacity": 0.5})

In [11]:
white.max_order


10